In [13]:
from libs.my.dataloader import DataLoader
from libs.deeplab.model import Deeplabv3, preprocess_input
import cv2

ModuleNotFoundError: No module named 'keras'

In [ ]:
python -c "import tensorflow as tf; tf.enable_eager_execution(); print(tf.reduce_sum(tf.random_normal([1000, 1000])))"

In [2]:
video_path = './data/' + '2.mp4'
dataloader = DataLoader(video_path, batch_size = 4)

In [3]:
ret, batch = dataloader.__get_batch__()

In [4]:
len(dataloader)

409

In [5]:
batch.shape

(4, 1080, 1920, 3)

In [6]:
from libs.ssd.vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd, create_mobilenetv1_ssd_predictor

In [7]:
model_path = './weights/mobilenet-v1-ssd-mp-0_675.pth'

In [8]:
net = create_mobilenetv1_ssd(21, is_test=True)
net.load(model_path)
predictor = create_mobilenetv1_ssd_predictor(net, candidate_size=200)

In [9]:
def calculate_boxes(boxes):
    # y1, x1, y2, x2
    # slide calculate
    if boxes[0] > 1080-boxes[2]:
        slide = boxes[0]
    else:
        slide = 1080-boxes[2]
    
    center = ((boxes[2]- boxes[0])/2 + boxes[0], (boxes[3]- boxes[1])/2 + boxes[1])
    return center, slide

def cut_tile(frame, center, subsize):
    slide = subsize / 2
    tile = frame[center[0]-slide:center[0]+slide, center[1]-slide:center[1]+slide]
    
    return tile

def batch_to_tiles(batch, boxes, batch_n, center_upper, center_bottom, subsize=512):
    upper_tiles = [] 
    bottom_tiles = []
    
    for i in range(0, batch.shape[0]):
        frame_n = i + batch.shape[0] * batch_n
        
        if frame_n % 30 == 0: 
            center, slide = calculate_boxes(boxes[i][0])
            center_upper = (center[0]-slide, center[1])
            center_bottom = (center[0]+slide, center[1])
        
            if center_upper - subsize < 0:
                slide_up = center_upper - subsize
                center_upper[0] -= slide_up
            
            if center_bottom + subsize > 1080:
                slide_up = 1080 - (center_bottom + subsize)
                center_bottom[0] += slide_up
        
        #upper_tiles.append(cut_tile(batch[i], center_upper, subsize))
        #bottom_tiles.append(cut_tile(batch[i], center_bottom, subsize))
        if i == 0:
            upper_batch = cut_tile(batch[i], center_upper, subsize)[None]
            bottom_batch = cut_tile(batch[i], center_bottom, subsize)[None]
        else:
            upper_batch = np.concatenate((upper_batch, cut_tile(batch[i], center_upper, subsize)[None]) , axis=0)
            bottom_batch = np.concatenate((bottom_batch, cut_tile(batch[i], center_bottom, subsize)[None]) , axis=0)
            
        
    return upper_batch, bottom_batch, center_upper, center_bottom

In [10]:
class Predictor:
    def __init__(self, detector, net):
        self.detector = detector
        self.net = net
        #self.dataloader = None

    def predict(self, dataloader):
        """
            1. Получить батч
                1.1 Препроцессинг батч
            2. Задетектить боксы в батче
                2.1 Препроцессинг батч
            3. Взять маску
                3.1 Постпроцессинг батч
        """
        
        preds = []
        ret = True
        i = 0
        center_upper, center_bottom = None, None
        while ret:
            ret, batch = dataloader.__get_batch__()
            # boxes, labels, probs = self.detector.detect_batch(batch)
            boxes, labels, probs = self.__detect_batch(batch)
            upper_batch, bottom_batch, center_upper, center_bottom \
            = batch_to_tiles(batch, boxes, i, center_upper, center_bottom)
            
            upper_batch = preprocess_input(upper_batch)
            bottom_batch = preprocess_input(bottom_batch)
            
            upper_preds = self.net.predict(upper_batch)
            bottom_preds = self.net.predict(bottom_batch)
            
            i += 1
        return upper_preds, upper_preds
            
    def __detect_batch(self, batch):
        boxes_ = []
        labels_ = []
        probs_ = []
        
        for i in range(0, batch.shape[0]):
            print(i)
            image = cv2.cvtColor(batch[i], cv2.COLOR_BGR2RGB)
            boxes, labels, probs = self.detector.predict(image, 10, 0.4)
            boxes_.append(boxes)
            labels_.append(labels)
            probs_.append(probs)
            
        return boxes_, labels_, probs_        

In [11]:
model = Predictor(predictor, None)

In [12]:
batch.shape

(4, 1080, 1920, 3)